### OpenFIBSEM DB

sqlite3 database for storing metadata about FIBSEM datasets.

https://www.sqlitetutorial.net/sqlite-python

DATABASE

--- EXPERIMENT MANAGMENT ---

project
- id
- name
- user_id

user:
- id
- name

experiment
- id
- name
- project_id
- user_id
- sample_id
- path

sample
- id
- name



---- Analytics ----

ml
detection
alignment
interaction
history
steps


In [ ]:

# MILESTONE 1
# TODO: initialise database script
# TODO: migrate ml data collection to store in db
# TODO: setup projects, experiment, user tables
# TODO: separate ETL from statistics / analytics application
# TODO: enable user to select project, experiment
# TODO: enable multi-experiment analytics
# TODO: fix experiment to dataframe function, make actually useful


In [ ]:
%load_ext autoreload
%autoreload 2

import datetime
import sqlite3
from fibsem.db.util import *
from fibsem.db.util import _create_database
from fibsem import config as cfg


### CREATE DATABASE / TABLES

In [ ]:
_create_database()

### Create Project

In [ ]:
import datetime
database = cfg.DATABASE_PATH
conn = create_connection(database)
# create_project(conn, 
#     ("WAFFLE-METHOD-DEVELOPMENT", datetime.datetime.now(), "patrick")
# )

create_project(conn, 
    ("AUTOLIFTOUT-METHOD-DEVELOPMENT", datetime.datetime.now(), "patrick")
)

### CREATE USER

In [ ]:
create_user(conn, 
    ('hannah', 'hannah.siems@monash.edu', 'password')
)

### CREATE SAMPLE

In [ ]:
create_sample(
    conn, 
    ("WAFFLE-01", 1, datetime.datetime.now(), "hannah")
)

### CREATE EXPERIMENT


In [ ]:
create_experiment(
    conn, 
    ("WAFFLE-01", 1, datetime.datetime.now(), "hannah", 1, "autolamella", "waffle", "")
)

In [ ]:
import pandas as pd
print("PROJECTS")
df = pd. read_sql('SELECT * from projects', conn)
display(df)

print("USERS")
df = pd. read_sql('SELECT * from users', conn)
display(df)

print("SAMPLES")
df = pd. read_sql('SELECT * from samples', conn)
display(df)

print("EXPERIMENTS")
df = pd. read_sql('SELECT * from experiments', conn)
display(df)

In [ ]:
### REPLACE ? --> UPSERT?

# might not want to use, replaces experiment?

### Sync Experiments


In [ ]:

PROJECT_NAME = "WAFFLE-METHOD-DEVELOPMENT"

df = pd.read_sql(f"SELECT * FROM projects WHERE name='{PROJECT_NAME}'", conn)
PROJECT_IDS = df["id"].values
print(PROJECT_IDS)
df = pd.read_sql(f"SELECT * FROM experiments WHERE project_id={PROJECT_IDS[0]}", conn)

display(df)

#### Upload all data to Database

In [ ]:
%load_ext autoreload
%autoreload 2
from autolamella import config as cfg
import os

import pandas as pd

from autolamella.tools.data import calculate_statistics_dataframe


import datetime
import sqlite3
from fibsem.db.util import *





def _update_experiment(conn, PATH:str, _UPDATE: bool, program: str = "autolamella", method: str = "waffle", user: str = "hannah", project_id: int = 1, sample_id: int = 1):
    
    """"""
    print(PATH)   
    print("-------EXPERIMENT-------")

    if program == "autolamella":
        from autolamella.structures import Experiment
    if program == "autoliftout":
        from liftout.structures import Experiment
    exp = Experiment.load(os.path.join(PATH, "experiment.yaml"))
    df = exp.to_dataframe_v2()

    # format date as datetime
    df["date"] = pd.to_datetime(df["date"], unit="s")
    df["date"] = pd.to_datetime(df["date"], utc=True)

    # drop experiment_id
    df = df.drop(columns=["experiment_id", "num_lamella"])
    df["project_id"] = project_id
    df["sample_id"] = sample_id
    df["user"] = user
    df["method"] = method
    
    display(df)

    ret  = input("continue?")
    
    if "n" in ret.lower():
        return

    if _UPDATE:
        df.to_sql('experiments', conn, if_exists='append', index = False)

    print("------------------")

    print("-----------DATABASE EXPERIMENTS -------")
    df_exp = pd.read_sql("SELECT * FROM experiments", conn)
    display(df_exp)
    print("------------------")

    # return

    # break

    dfs = calculate_statistics_dataframe(PATH, program=program)

    print("-------HISTORY----------")
    df = dfs[1]

    # format date as datetime
    df["start"] = pd.to_datetime(df["start"], unit="s")
    df["start"] = pd.to_datetime(df["start"], utc=True)

    df["end"] = pd.to_datetime(df["start"], unit="s")
    df["end"] = pd.to_datetime(df["start"], utc=True)


    EXP_ID = df_exp[df_exp["name"]==df["exp_name"].iloc[0]]["id"].iloc[0]

    df["experiment_id"] = EXP_ID
    # drop experiment_id
    df = df.drop(columns=["exp_id", "exp_name"])
    
    display(df.head())

    if _UPDATE:
        df.to_sql('history', conn, if_exists='append', index = False)

    print("------------------")


    print("-------STEPS----------")

    df = dfs[3]
    # display(df)
    # # format date as datetime
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")
    df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True)

    EXP_ID = df_exp[df_exp["name"]==df["exp_name"].iloc[0]]["id"].iloc[0]

    df["experiment_id"] = EXP_ID
    # drop experiment_id
    df = df.drop(columns=["exp_id", "exp_name"])

    # rename lamella to petname
    df = df.rename(columns={"lamella": "petname"})

    # replace NA with 0
    df = df.fillna(0)
    
    display(df.head())
    
    if _UPDATE:
        df.to_sql('steps', conn, if_exists='append', index = False)

    print("------------------")


    print("-------DETECTIONS----------")
    df = dfs[5]


    # # # format date as datetime
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")
    df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True)

    EXP_ID = df_exp[df_exp["name"]==df["exp_name"].iloc[0]]["id"].iloc[0]

    df["experiment_id"] = EXP_ID
    # # drop experiment_id
    df = df.drop(columns=["exp_id", "exp_name"])

    # # rename lamella to petname
    df = df.rename(columns={"lamella": "petname"})

    # # replace NA with 0
    df = df.fillna(0)
    
    display(df.head())
    
    if _UPDATE:
        df.to_sql('detections', conn, if_exists='append', index = False)

    print("------------------")


    #### INTERACTIONS
    print("-------INTERACTIONS----------")
    df = dfs[6]

    # # # format date as datetime
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")
    df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True)

    EXP_ID = df_exp[df_exp["name"]==df["exp_name"].iloc[0]]["id"].iloc[0]


    df["experiment_id"] = EXP_ID
    # # drop experiment_id
    df = df.drop(columns=["exp_id", "exp_name"])

    # # rename lamella to petname
    df = df.rename(columns={"lamella": "petname"})

    # # replace NA with 0
    df = df.fillna(0)
    
    display(df.head())
    
    if _UPDATE:
        df.to_sql('interactions', conn, if_exists='append', index = False)

    print("------------------")


In [ ]:
# create / connect to db
database = "fibsem.db"
conn = create_connection(database)


_UPDATE = True

EXPERIMENTS = [
    "/home/patrick/github/data/EXPERIMENTS/HANNAH-WAFFLE-01-07092023",
    "/home/patrick/github/data/EXPERIMENTS/HANNAH-WAFFLE-01-12092023",
    "/home/patrick/github/data/EXPERIMENTS/HANNAH-WAFFLE-02-13092023",
    "/home/patrick/github/data/EXPERIMENTS/HANNAH-WAFFLE-01-20092023",
    "/home/patrick/github/data/EXPERIMENTS/HANNAH-WAAFFLE-01-21092023",
    ]
# AUTOLIFTOUT_EXPERIMENTS = ["/home/patrick/github/data/EXPERIMENTS/AUTOLIFTOUT-WAFFLE-01-24082023"]

for PATH in EXPERIMENTS:
    _update_experiment(conn, PATH, _UPDATE, program="autolamella", method="waffle", user="hannah")
    
# for PATH in AUTOLIFTOUT_EXPERIMENTS:
#     _update_experiment(conn, PATH, _UPDATE, program="autoliftout", method="autoliftout", user="patrick")

#### New Logging



In [4]:
%load_ext autoreload 
%autoreload 2
from autolamella.tools.data import calculate_statistics_dataframe


PATH = "/home/patrick/github/autolamella/autolamella/log/AutoLiftout-2024-01-15-14-33"
dfs = calculate_statistics_dataframe(PATH, program="autolamella", encoding="utf-8")


print('-'*80)
print(f"Experiments: {len(dfs[0])}")
# # experiment
display(dfs[0])

print('-'*80)
print(f"History: {len(dfs[1])}")
# # history
display(dfs[1])

# print('-'*80)
# print(f"Beam shifts: {len(dfs[2])}")
# # # beam_shift
# display(dfs[2])

# print('-'*80)
# print(f"Steps: {len(dfs[3])}")
# # # steps
# display(dfs[3])

# print('-'*80)
# print(f"Detections: {len(dfs[5])}") 
# # detections
# display(dfs[5])

# print('-'*80)
# print(f"Interactions: {len(dfs[6])}")
# # # interactions
# display(dfs[6])


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
--------------------------------------------------------------------------------
Parsing /home/patrick/github/autolamella/autolamella/log/AutoLiftout-2024-01-15-14-33/logfile.log
--------------------------------------------------------------------------------
Experiments: 2


,experiment_name,experiment_path,experiment_created_at,experiment_id,program,method,number,petname,path,lamella.x,lamella.y,lamella.z,lamella.r,lamella.t,last_timestamp,current_stage,exp_name
0,AutoLiftout-2024-01-15-14-33,/home/patrick/github/autolamella/autolamella/l...,1.705290e+09,c8fbe436-880b-4676-be30-0714e21df756,AutoLiftout,AutoLiftout,1,01-vast-fawn,/home/patrick/github/autolamella/autolamella/l...,0.002411,-0.002524,0.031841,-2.286332,0.296702,1.705290e+09,LiftoutLamella,AutoLiftout-2024-01-15-14-33
1,AutoLiftout-2024-01-15-14-33,/home/patrick/github/autolamella/autolamella/l...,1.705290e+09,c8fbe436-880b-4676-be30-0714e21df756,AutoLiftout,AutoLiftout,2,02-true-fox,/home/patrick/github/autolamella/autolamella/l...,0.002411,-0.002524,0.031841,-2.286332,0.296702,1.705290e+09,Finished,AutoLiftout-2024-01-15-14-33


--------------------------------------------------------------------------------
History: 12


,petname,stage,start,end,duration,exp_name,exp_id
0,01-vast-fawn,SetupTrench,1.705290e+09,1.705290e+09,15.588577,AutoLiftout-2024-01-15-14-33,c8fbe436-880b-4676-be30-0714e21df756
1,01-vast-fawn,ReadyTrench,1.705290e+09,1.705290e+09,0.000031,AutoLiftout-2024-01-15-14-33,c8fbe436-880b-4676-be30-0714e21df756
2,01-vast-fawn,MillTrench,1.705290e+09,1.705290e+09,5.629104,AutoLiftout-2024-01-15-14-33,c8fbe436-880b-4676-be30-0714e21df756
3,01-vast-fawn,MillTrench,1.705290e+09,1.705290e+09,20.147232,AutoLiftout-2024-01-15-14-33,c8fbe436-880b-4676-be30-0714e21df756
4,01-vast-fawn,MillUndercut,1.705290e+09,1.705290e+09,0.000921,AutoLiftout-2024-01-15-14-33,c8fbe436-880b-4676-be30-0714e21df756
5,01-vast-fawn,LiftoutLamella,1.705290e+09,1.705290e+09,140.880428,AutoLiftout-2024-01-15-14-33,c8fbe436-880b-4676-be30-0714e21df756
6,02-true-fox,LandLamella,1.705290e+09,1.705290e+09,143.640691,AutoLiftout-2024-01-15-14-33,c8fbe436-880b-4676-be30-0714e21df756
7,02-true-fox,SetupLamella,1.705290e+09,1.705290e+09,22.857306,AutoLiftout-2024-01-15-14-33,c8fbe436-880b-4676-be30-0714e21df756
8,02-true-fox,ReadyLamella,1.705290e+09,1.705290e+09,0.000062,AutoLiftout-2024-01-15-14-33,c8fbe436-880b-4676-be30-0714e21df756
9,02-true-fox,MillRoughCut,1.705290e+09,1.705290e+09,6.376390,AutoLiftout-2024-01-15-14-33,c8fbe436-880b-4676-be30-0714e21df756
